In [54]:
import requests
import json
from psycopg2 import connect

In [74]:
def get_variables(data):
    google_id = data['google_id']
    #identifier = data['result']['name'].replace(" ", "").lower()
    restaurant_name = data['result']['name']
    try:
        address = data['result']['formatted_address'].replace("'", "")
    except:
        address = 'no address'
    try:
        postcode = data['result']['address_components'][6]['long_name']
    except:
        postcode = 'no postcode'
    try:
        website = data['result']['website']
    except:
        website = 'no website'
    try:
        phone_number = data['result']['formatted_phone_number']
    except:
        phone_number = 'no phone number'
    try:
        opening_hours = str(data['result']['opening_hours']['weekday_text'])
    except:
        opening_hours = 'no opening hours'
    try:
        rating = data['result']['rating']
    except:
        rating = 'no rating'
    try:
        total_ratings = data['result']['user_ratings_total']
    except:
        total_ratings = 'no total ratings'
    try:
        price_level = data['result']['price_level']
    except:
        price_level = 'no price level'
    
    
    return google_id, restaurant_name, address, postcode, website, phone_number, rating, total_ratings, price_level

In [76]:
def etl(restaurant):
    url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={restaurant}%20London&inputtype=textquery&fields=place_id&key=apikey"
    
    payload={}
    headers = {}
    response = requests.get(url, headers=headers, data=payload)

    restaurant_id = json.loads(response.text)
    rest = restaurant_id['candidates'][0]['place_id']
    
    url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={rest}&key=apikey"
    
    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    
    json_data = response.json()
    
    google_id = {'google_id' : rest}
    
    json_data.update(google_id)
    
    table_name = 'public.google'
    sql_string = f'INSERT INTO {table_name}\nVALUES {get_variables(json_data)};'

    conn = connect(
        dbname = "initialdatabase",
        user = "username",
        host = "demsin0166.czfwea5noxbs.eu-west-2.rds.amazonaws.com",
        password = "password")
    
    cur = conn.cursor()
    
    try:
        cur.execute( sql_string )
        conn.commit()
        print(f'{restaurant} added to database')
    except:
        print(f'{restaurant} already in database')
    
    cur.close()
    conn.close()
    

In [72]:
res_list = ['Bonoo Indian Tapas', 'Hibox', 'Scarlett Green', 'Devine Restaurant Coffee Bar', 'Indian Room', 'Nora Cafe', 'Amrutha Lounge', 'Bayleaf Restaurant', 'Andy’s Greek Taverna', 'Alexander the Great', 'Chamisse', 'Taste Of Nawab', 'Muhib Indian Cuisine', 'Spasso', 'Awesome Thai Finchley', 'Indian Moment', 'Bruce Burgers', 'Tania’s of Hampstead', 'Cafe Parisienne', 'The Rajdoot']

In [73]:
for i in res_list:
    etl(i)

Bonoo Indian Tapas already in database
Hibox already in database
Scarlett Green already in database
Devine Restaurant Coffee Bar already in database
Indian Room already in database
Nora Cafe already in database
Amrutha Lounge already in database
Bayleaf Restaurant already in database
Andy’s Greek Taverna already in database
Alexander the Great already in database
Chamisse already in database
Taste Of Nawab already in database
Muhib Indian Cuisine already in database
Spasso already in database
Awesome Thai Finchley already in database
Indian Moment already in database
Bruce Burgers already in database
Tania’s of Hampstead already in database
Cafe Parisienne already in database
The Rajdoot already in database


In [79]:
etl("Goldmine")

Goldmine added to database
